We'll do a simple greenscreen replacement with a machine-learning model.

In the following code cells, we will:
- download and display an image
- pre-process the pixels, labeling background and foreground pixels
- train a model to identify green background pixels
- use the predictions of the model to swap the green screen background for an image of a forest

In [ ]:
import numpy as np
# from keras.preprocessing import image
import tensorflow as tf
from tensorflow import keras

Display the image using Keras

In [ ]:
# img is an object of type PIL.Image.Image.
img = keras.utils.load_img("imgs/greenML.png")

# display is a built-in IPython function.
display(img)

Display a cropped version of the image

In [ ]:
# arr is np.ndarray of shape height * width * 3, i.e., arr.shape == (713, 1215, 3)
arr = keras.utils.img_to_array(img)
# Trim off edges
# Original code uses arr[:697,:], but that isn't quite complete, such that at the
# bottom of the cropped image, there is one row of dark gray pixels (color (52, 52, 52)).
arr = arr[:696,:]
display(tf.keras.preprocessing.image.array_to_img(arr,scale=False))

The following few cells extract the background and foreground colors from the image, but in the end the extracted information is not used. Instead, the final block of code uses hardcoded background color.

Isolate the background and make a dataset of background pixels, `YesSet`.

In [ ]:
# background
tmp = arr[:,:360]
display(tf.keras.preprocessing.image.array_to_img(tmp,scale=False))

YesSet = np.reshape(tmp,(-1,3))

Print the number dimensions of `tmp`.

In [ ]:
tmp.shape

Print the dimensions of `YesSet`.



In [ ]:
YesSet.shape

Build `seen` dictionary of the unique pixel colors in `yes_list`.  For all keys in the dictionary, the value is 1.

In [ ]:
seen = {}
for c in YesSet:
  # The original code converts `c` to str before inserting into `seen`,
  # and is quite slow. We achieve a major speed-up by using tuple instead.
  seen[tuple(c.tolist())] = 1
# Convert from tuple to string, to be compatible with the original code.
seen = {str(np.array(c)): 1 for c in seen.keys()}
len(seen)

Isolate part of the foreground and make a dataset of foreground pixels called `NoSet`.

In [ ]:
# foreground
tmp = arr[30:,547:620]
display(tf.keras.preprocessing.image.array_to_img(tmp,scale=False))

NoSet = np.reshape(tmp,(-1,3))

Dimensions of the full image, corresponding to the height, width, and RGB (red, green, blue) colors.

In [ ]:
arr.shape

We finalize our dataset, with a variable `alldat` cointaining our list of pixels, and `labs` holding our list of labels for each pixel (`0` for green background pixels and `1` for foreground pixels)

In [ ]:
# Build a list of pixels for both positive and negative examples.
alldat = np.concatenate((YesSet,NoSet))
 
# labels
labs = np.concatenate((np.ones(len(YesSet)), np.zeros(len(NoSet))))

We display the image of the forest, `img` and covert the image into an array, `bkg`.

In [ ]:
img = keras.utils.load_img("imgs/forest.jpg")

display(img)

bkg = keras.utils.img_to_array(img)

Define drawScreen for merging foreground and background.

In [ ]:
# Contains some perf optimizations, reducing the run time of the following code
# block from 13 seconds to 5 seconds.

def distance_square(c1: np.ndarray, c2: np.ndarray):
  return ((c1 - c2) ** 2).sum()

def drawScreen(c: np.ndarray, d: float,
               studio_image: np.ndarray, background_image: np.ndarray):
  display_image = studio_image.copy()
  for x in range(min(background_image.shape[0],studio_image.shape[0])):
    for y in range(min(background_image.shape[1], studio_image.shape[1])):
      if distance_square(c, studio_image[x][y]) <= d:
        display_image[x][y] = background_image[x][y]
  display(tf.keras.preprocessing.image.array_to_img(display_image,scale=False))

Put ML into the forest.

In [ ]:
drawScreen(np.array([0.3,174.5,46.7]), 1000.0, arr, bkg)